In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import wandb
from wandb.integration.keras import WandbCallback

wandb.login(key="2a6d75e62c637b3c3f6f727ad4ebe3e85efe85be")
 
# Load dataset
df = pd.read_csv("smaller_dataset.csv")
 
# Preprocess the dataset
df_encoded = df.drop(columns=[df.columns[0]])  # Drop the first column
X = df_encoded.drop(columns=["DRUH_POVR", "NAZ_LOKALI"])  # Features
y = df_encoded["DRUH_POVR"]  # Target
 
# Convert features to TensorFlow tensors
X_tensor = tf.convert_to_tensor(X.to_numpy(), dtype=tf.float32)
 
# Use TensorFlow StringLookup for label encoding
label_lookup = tf.keras.layers.StringLookup(output_mode='int', vocabulary=tf.constant(y.unique()))
y_tensor = label_lookup(y) - 1  # Adjust labels to start from 0
 
# Create a TensorFlow Dataset
num_classes = len(label_lookup.get_vocabulary()) - 1  # Adjust for 0-based indexing
y_one_hot = tf.one_hot(y_tensor, depth=num_classes)
dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_one_hot))
 
# Shuffle the dataset
dataset_size = len(y_tensor)
dataset = dataset.shuffle(buffer_size=dataset_size, seed=42)
 
# Split the dataset into train, validation, and test sets
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size
 
train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)
 
# Batch the datasets for training, validation, and evaluation
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
 
# Determine the number of classes for the output layer
# num_classes = len(label_lookup.get_vocabulary()) - 1  # Adjust for 0-based indexing
y_encoded = y_tensor.numpy()
y_categorical = to_categorical(y_encoded, num_classes=num_classes)
#  Define the model
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),  # Specify input shape
#     tf.keras.layers.Dense(num_classes, activation='softmax'),  # Output layer
# ])
 
 
 
# Compile the model
# model.compile(
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
#     optimizer=tf.keras.optimizers.Adam(),
#     metrics=['accuracy']  # Add accuracy as a metric
# )

wandb.init(
    project="Test-project-mik",  # Change this to your project name
    name="test-project-mik",  # Give a meaningful name
    config={  # Log hyperparameters
        "epochs": 10,
        "batch_size": batch_size,
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "architecture": "Conv1D",
        "num_classes": num_classes
    }
)
 
input_shape=(*X.shape[1:],1)
filters = [64, 32, 16]
kernel_sizes=[9 for i in range(4)]
padding='causal'
print(y.shape)
model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Input(
        shape=input_shape
    )
)
model.add(
    tf.keras.layers.BatchNormalization()
)
 
for f,k in zip(filters, kernel_sizes):
    model.add(
        tf.keras.layers.Conv1D(
            filters=f,
            kernel_size=k,
            activation=tf.nn.relu,
            padding=padding,
            kernel_regularizer='l2'
        )
    )
    model.add(
        tf.keras.layers.BatchNormalization()
    )
    model.add(tf.keras.layers.ReLU())
 
model.add(tf.keras.layers.GlobalAveragePooling1D())        
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)
 
# Train the model with a validation set
history = model.fit(
    train_dataset, 
    validation_data=val_dataset,  # Include validation set
    epochs=10,  # Increased epochs for better training
    verbose=1,  # Display progress
    callbacks=[WandbCallback()]  # Log metrics automatically
)
 
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_accuracy
})

wandb.finish()  # End the wandb run


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hucko-miso (hysped) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
2025-02-26 07:58:51.965525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-26 07:58:51.981943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:

wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


(32950,)
Epoch 1/10


2025-02-26 07:58:53.971547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2025-02-26 07:58:54.200934: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-02-26 07:58:54.212580: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x728ed03c1d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-26 07:58:54.212665: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-02-26 07:58:54.217888: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-26 07:58:54.309521: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of th

721/721 [==============================] - ETA: 0s - loss: 2.4342 - categorical_accuracy: 0.3090   

...

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /workspace/first-experiment/wandb/run-20250226_075852-4gl6i6x7/files/model-best/assets


INFO:tensorflow:Assets written to: /workspace/first-experiment/wandb/run-20250226_075852-4gl6i6x7/files/model-best/assets
wandb: Adding directory to artifact (/workspace/first-experiment/wandb/run-20250226_075852-4gl6i6x7/files/model-best)... Done. 0.0s


721/721 [==============================] - 5s 7ms/step - loss: 1.8262 - categorical_accuracy: 0.4435 - val_loss: 1.8059 - val_categorical_accuracy: 0.4516
Epoch 7/10
721/721 [==============================] - 4s 5ms/step - loss: 1.7786 - categorical_accuracy: 0.4573 - val_loss: 2.0638 - val_categorical_accuracy: 0.3768
Epoch 8/10
721/721 [==============================] - 4s 6ms/step - loss: 1.7506 - categorical_accuracy: 0.4659 - val_loss: 2.2732 - val_categorical_accuracy: 0.3383
Epoch 9/10
721/721 [==============================] - 4s 5ms/step - loss: 1.7168 - categorical_accuracy: 0.4826 - val_loss: 1.9445 - val_categorical_accuracy: 0.4270
Epoch 10/10
721/721 [==============================] - 4s 5ms/step - loss: 1.6822 - categorical_accuracy: 0.4856 - val_loss: 2.0726 - val_categorical_accuracy: 0.4019
Test Loss: 2.1082, Test Accuracy: 0.3856


categorical_accuracy,▁▃▄▅▆▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▃▂▂▂▁▁
test_accuracy,▁
test_loss,▁
val_categorical_accuracy,▁▅▆▆▅█▅▄▇▆
val_loss,█▃▂▂▃▁▃▆▂▄
best_epoch,5
best_val_loss,1.80593
categorical_accuracy,0.48563
epoch,9


In [11]:
y.shape

(32950,)

In [15]:
len(y.unique())

18

In [2]:
train_dataset.

AttributeError: '_BatchDataset' object has no attribute 'shape'

In [1]:
import tensorflow as tf

# List all physical devices of type 'GPU'
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are available:", gpus)
else:
    print("No GPU found. Using CPU.")

2025-02-18 07:42:30.536476: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 07:42:30.571586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-18 07:42:31.133608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPUs are available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-02-18 07:42:31.669760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-18 07:42:31.751669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-18 07:42:31.751827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf